Question No 1: Data Processing

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd

In [4]:
carsData = pd.read_csv(Path("Cars93.csv"))

In [12]:
carsData.columns

Index(['Manufacturer', 'Model', 'Type', 'Min.Price', 'Price', 'Max.Price',
       'MPG.city', 'MPG.highway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile', 'Man.trans.avail',
       'Fuel.tank.capacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turn.circle', 'Rear.seat.room', 'Luggage.room', 'Weight', 'Origin'],
      dtype='object')

In [15]:
carsData.loc[:,["Model","Type","Max.Price","AirBags"]].head()

,Model,Type,Max.Price,AirBags
0,Integra,Small,18.8,NaN
1,Legend,Midsize,38.7,Driver & Passenger
2,90,Compact,32.3,Driver only
3,100,Midsize,44.6,Driver & Passenger
4,535i,Midsize,36.2,Driver only


Q.No 1.1: Assign a type to each of the following features (a) Model, (b) Type, (c) Max. Price and
(d) Airbags from the following: ordinal/nominal/ratio/interval scale.
Ans:

| Features   | Type    |
|----------------------|
| Model      |         |
| Type       |         |
| Max.Price  |         |  
| AirBags    |         |  
|____________|_________|
